In [1]:
# pip install transformers
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.5 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=6354b10ad0ce4657f6d33d83fb8fb9b299d7f5fe35aaccc2d18327366b627625
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00


In [3]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
train_arg = False
startDate = '2021-01-01'
endDate = '2021-04-01'

import torch
import pandas as pd
import datetime
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

# from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AdamW, get_scheduler
from torch.utils.data import DataLoader

from time import time

# TRAIN OR EVAL
train = train_arg
trained_model_name = "model_covid_twitter_bert_v2.model"

# MODEL DEFINITION
if train:
    # BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert"
    # BASE_MODEL = "./covid-twitter-bert"
    BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert-v2"
    # BASE_MODEL = "./covid-twitter-bert-v2"
else:
    BASE_MODEL = "/content/" + trained_model_name
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
# BATCH_SIZE = 16
BATCH_SIZE = 2
EPOCHS = 20

# accelerator = Accelerator()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)
optimizer = AdamW(model.parameters(), lr=3e-5)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(torch.cuda.is_available())

print(torch.cuda.device_count())

# print(torch.cuda.current_device())

# print(torch.cuda.device(0))

# print(torch.cuda.get_device_name(0))

# DATA PREPARATION

g_cases_filename = f"/content/data/g_cases_2021.csv"
y = pd.read_csv(g_cases_filename)

X = pd.DataFrame()
# date_ranges = [['2021-01-01', '2021-04-01']]
# date_ranges = [['2020-01-01', '2020-01-12']]
date_ranges = [[startDate, endDate]]
for date_range in date_ranges:
    start = datetime.datetime.strptime(date_range[0], "%Y-%m-%d")
    end = datetime.datetime.strptime(date_range[1], "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

    for date in date_generated:
        date_str = date.strftime("%Y-%m-%d")

        filename = f"/content/data/en_{date_str}_output.csv"
        df_data = pd.read_csv(filename)
        df_data = df_data.replace(r'^\s*$', np.nan, regex=True)
        #X = total_dataframe.append(df_data, ignore_index=True)
        X = pd.concat([X, df_data], ignore_index=True)

print(f"X.shape: {X.shape}")
print(f"X.head(): {X.head()}")

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set, test size 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=42, shuffle=False)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42, shuffle=False)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42, shuffle=False)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("train_df")
print(f"train_df.shape: {train_df.shape}")
print(train_df)

print("val_df")
print(f"val_df.shape: {val_df.shape}")
print(val_df)

print("test_df")
print(f"test_df.shape: {test_df.shape}")
print(test_df)

raw_train_ds = Dataset.from_pandas(train_df)
raw_val_ds = Dataset.from_pandas(val_df)
raw_test_ds = Dataset.from_pandas(test_df)

max_length = tokenizer.model_max_length
print("Maximum input sequence length:", max_length)
# Maximum input sequence length: 1000000000000000019884624838656

ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

print("ds")
print(ds)

# CALCULATE MAX_LENGTH
# Tokenize all tweets to find the maximum tokenized length
# max_length = 0

# for index, row in X.iterrows():
#     tweet = row['clean_tweets']

#     # Tokenize the tweet
#     tokens = tokenizer(tweet, return_tensors="pt")

#     # Get the length of the tokenized sequence
#     length = tokens['input_ids'].shape[1]
#     print(f"current length: {length}")

#     # Update max_length if needed
#     if length > max_length:
#         max_length = length

# print(f"Calculated max_length: {max_length}")
# Model's max length
# max_length = 512
# RuntimeError: The size of tensor a (600) must match the size of tensor b (512) at non-singleton dimension 1

def preprocess_function(examples):
    label = examples["g_values"]
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length", max_length=256)
    # examples = tokenizer(examples["clean_tweets"], truncation=False, padding=True, return_tensors="pt")
    examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length")

    # examples = tokenizer(examples["clean_tweets"])

    examples["label"] = float(label)
    # examples["label"] = [float(i) for i in label]
    print(examples)
    return examples

from torch.utils.data import DataLoader

if train:
    ds["train"] = ds["train"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])
    ds["validation"] = ds["validation"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

    train_dataloader = DataLoader(
        ds["train"], shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator
    )
    eval_dataloader = DataLoader(
        ds["validation"], batch_size=BATCH_SIZE, collate_fn=data_collator
    )
else:
    ds["test"] = ds["test"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

    test_dataloader = DataLoader(
        ds["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
    )

# for split in ds:
    # ds[split] = ds[split].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])

print("ds")
print(ds)

# for batch in train_dataloader:
    # batch = {k: v.to(device) for k, v in batch.items()}
    # break
# print({k: v.shape for k, v in batch.items()})

# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

def training():
    num_training_steps = EPOCHS * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    print(f"num_training_steps: {num_training_steps}")

    from tqdm.auto import tqdm

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, batch in enumerate(train_dataloader):
            print(f"epoch: {epoch}, batch: {batch_idx}")
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            # accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

def evaluation(dataloader):
    eval_preds = []
    eval_labels = []
    model.eval()
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        print(f"outputs['logits'].shape: {outputs['logits'].shape}")
        eval_preds.append(outputs['logits'])

        print(f"batch['labels'].shape: {batch['labels'].shape}")
        eval_labels.append(batch['labels'])

    eval_preds = torch.cat(eval_preds, dim=0)
    print(f"eval_preds: {eval_preds}")
    eval_labels = torch.cat(eval_labels, dim=0)
    print(f"eval_labels: {eval_labels}")
    metrics = compute_metrics_for_regression(eval_preds, eval_labels)
    print(metrics)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred, eval_labels):
    print(f"eval_pred: {eval_pred}")
    print(f"eval_labels: {eval_labels}")
    logits = eval_pred
    labels = eval_labels
    labels = torch.reshape(labels, (-1, 1))
    print(f"labels: {labels}")

    mse = mean_squared_error(labels.cpu(), logits.cpu())
    mae = mean_absolute_error(labels.cpu(), logits.cpu())
    r2 = r2_score(labels.cpu(), logits.cpu())
    single_squared_errors = ((logits.cpu() - labels.cpu()).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

if train:
    ## TRAINING
    t = time()

    training()
    evaluation(eval_dataloader)
    model.save_pretrained(trained_model_name, from_pt=True)
    tokenizer.save_pretrained(trained_model_name)

    print('Time to train model: {} mins'.format(round((time() - t) / 60, 2)))

else:
    ## EVALUATION
    t = time()
    evaluation(test_dataloader)

    print('Time to eval model: {} mins'.format(round((time() - t) / 60, 2)))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


True
1
X.shape: (90, 2)
X.head():    created_at                                       clean_tweets
0  2021-01-01  mf doom passed favourite shop covid casualty i...
1  2021-01-02  vocal role head covid task force told people w...
2  2021-01-03  love grandma burdensome think people overweigh...
3  2021-01-04  covid leave hospital facing staffing issue sta...
4  2021-01-05  covid protocol net star unavailable week detai...
train_df
train_df.shape: (62, 5)
    created_at                                       clean_tweets        date  \
0   2021-01-01  mf doom passed favourite shop covid casualty i...  2021-01-01   
1   2021-01-02  vocal role head covid task force told people w...  2021-01-02   
2   2021-01-03  love grandma burdensome think people overweigh...  2021-01-03   
3   2021-01-04  covid leave hospital facing staffing issue sta...  2021-01-04   
4   2021-01-05  covid protocol net star unavailable week detai...  2021-01-05   
..         ...                                           

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

{'input_ids': [101, 3909, 2710, 3073, 6947, 9398, 8382, 3231, 2765, 2604, 3462, 4553, 22950, 8398, 5658, 4276, 22088, 11368, 2098, 2343, 16985, 24014, 3672, 15121, 14132, 3478, 6960, 9424, 27860, 4297, 17572, 4928, 15011, 21122, 11533, 2522, 17258, 3433, 2111, 14753, 11451, 2522, 17258, 14855, 2497, 5396, 2318, 5926, 2304, 2111, 3404, 17404, 3369, 2057, 3726, 25216, 7272, 2317, 2304, 3424, 3567, 2595, 4895, 3995, 23062, 3085, 4357, 12484, 12849, 13344, 2050, 9413, 6371, 2304, 11376, 3189, 2522, 17258, 15050, 6545, 16464, 4945, 3460, 2703, 2175, 2361, 2375, 8571, 20221, 4909, 21887, 23350, 17404, 3019, 15403, 6526, 2147, 3946, 3044, 22950, 3514, 3006, 2709, 3671, 2522, 17258, 19428, 29464, 2050, 5157, 2180, 2102, 8980, 6090, 3207, 7712, 2095, 3930, 4030, 2522, 17258, 17404, 5219, 9432, 2233, 7610, 3693, 7484, 2592, 2389, 5219, 9298, 2080, 16184, 2319, 21414, 4654, 2102, 3693, 7396, 7411, 2522, 17258, 5604, 2194, 11891, 2571, 3231, 2354, 2338, 8814, 4819, 21887, 23350, 7691, 6410, 7374, 

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
outputs['logits'].shape: torch.Size([2, 1])
batch['labels'].shape: torch.Size([2])
eval_preds: tensor([[1.5733],
        [1.5728],
        [1.5728],
        [1.5728],
        [1.5725],
        [1.5726],
        [1.5726],
        [1.5732],
        [1.5729],
        [1.5735],
        [1.5736],
        [1.5732],
        [1.5727],
        [1.5728]], device='cuda:0')
eval_labels: tensor([0.3700, 0.3830, 0.3825, 0.8469, 0.3301, 0.2938, 0.3374, 0.4015, 0.4210,
        0.4160, 0.9590, 0.3629, 0.3159, 0.34

In [5]:
import sys, getopt
google_colab = True

if google_colab:
    root_path = "/content"
    train_arg = False
    startDate = '2021-01-01'
    endDate = '2021-04-01'
else:
    root_path = "."
    arguments = sys.argv[1:]
    short_options = "t:s:e:"
    long_options = ["train=", "startDate=", "endDate="]

    options, values = getopt.getopt(arguments, short_options, long_options)

    for o, v in options:
        print(f"Option is {o}. Value is {v}.")
        if o == "-t" or o == "--train":
            if v == "True":
                train_arg = True
            else:
                train_arg = False

        if o == "-s" or o == "--startDate":
            startDate = v
        if o == "-e" or o == "--endDate":
            endDate = v

import json
import torch
import pandas as pd
import datetime
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader

from time import time

# TRAIN OR EVAL
train = train_arg
trained_model_name = "model_covid_twitter_bert_v2_trainer.model"

# MODEL DEFINITION
if train:
    # BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert"
    # BASE_MODEL = "./covid-twitter-bert"
    # BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert-v2"
    if google_colab:
        BASE_MODEL = "digitalepidemiologylab/covid-twitter-bert-v2"
    else:
        BASE_MODEL = f"{root_path}/covid-twitter-bert-v2"
else:
    BASE_MODEL = "./" + trained_model_name
LEARNING_RATE = 2e-5
# MAX_LENGTH = 256
# BATCH_SIZE = 16
BATCH_SIZE = 2
EPOCHS = 20

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# DATA PREPARATION

g_cases_filename = f"{root_path}/data/g_cases_2021.csv"
y = pd.read_csv(g_cases_filename)

X = pd.DataFrame()
# date_ranges = [['2021-01-01', '2021-04-01']]
# date_ranges = [['2020-01-01', '2020-01-12']]
date_ranges = [[startDate, endDate]]
for date_range in date_ranges:
    start = datetime.datetime.strptime(date_range[0], "%Y-%m-%d")
    end = datetime.datetime.strptime(date_range[1], "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

    for date in date_generated:
        date_str = date.strftime("%Y-%m-%d")

        # filename = f"./data/en_{date_str}_output.csv"
        filename = f"{root_path}/tokenize_bert_output/tok_{date_str}_output.csv"
        df_data = pd.read_csv(filename)
        X = pd.concat([X, df_data], ignore_index=True)

print(f"X.shape: {X.shape}")
print(f"X.head(): {X.head()}")

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set, test size 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=42, shuffle=False)
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=42, shuffle=False)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42, shuffle=False)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("train_df")
print(f"train_df.shape: {train_df.shape}")
print(train_df)

print("val_df")
print(f"val_df.shape: {val_df.shape}")
print(val_df)

print("test_df")
print(f"test_df.shape: {test_df.shape}")
print(test_df)

raw_train_ds = Dataset.from_pandas(train_df)
raw_val_ds = Dataset.from_pandas(val_df)
raw_test_ds = Dataset.from_pandas(test_df)

max_length = tokenizer.model_max_length
print("Maximum input sequence length:", max_length)
# Maximum input sequence length: 1000000000000000019884624838656

ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

print("ds")
print(ds)

# CALCULATE MAX_LENGTH
# Tokenize all tweets to find the maximum tokenized length
# max_length = 0

# for index, row in X.iterrows():
#     tweet = row['clean_tweets']

#     # Tokenize the tweet
#     tokens = tokenizer(tweet, return_tensors="pt")

#     # Get the length of the tokenized sequence
#     length = tokens['input_ids'].shape[1]
#     print(f"current length: {length}")

#     # Update max_length if needed
#     if length > max_length:
#         max_length = length

# print(f"Calculated max_length: {max_length}")
# Model's max length
max_length = 512
# RuntimeError: The size of tensor a (600) must match the size of tensor b (512) at non-singleton dimension 1

def preprocess_function(examples):
    label = examples["g_values"]
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length", max_length=256)
    # examples = tokenizer(examples["clean_tweets"], truncation=False, padding=True, return_tensors="pt")
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length", max_length=max_length)
    # examples = tokenizer(examples["clean_tweets"], truncation=True, padding="max_length")

    # examples = tokenizer(examples["clean_tweets"])

    print("examples['tokenized_tweets']")
    print(examples["tokenized_tweets"])
    s = examples["tokenized_tweets"].replace("\'", "\"")
    examples = json.loads(s)


    examples["label"] = float(label)
    # examples["label"] = [float(i) for i in label]
    # print(examples)
    return examples

# for split in ds:
    # ds[split] = ds[split].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "clean_tweets"])
    # ds[split] = ds[split].map(preprocess_function, remove_columns=["date", "total_cases", "g_values", "created_at", "clean_tweets"], batched=True)

if train:
    ds["train"] = ds["train"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])
    ds["validation"] = ds["validation"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])
else:
    ds["test"] = ds["test"].map(preprocess_function, remove_columns=["__index_level_0__", "date", "total_cases", "g_values", "created_at", "tokenized_tweets"])

print("ds")
print(ds)


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}



from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./models/covid-twitter-bert-v2-fine-tuned-regression",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=10,
    save_total_limit = 5,
    # save_strategy="no",
    metric_for_best_model="mse",
    load_best_model_at_end=True,
    weight_decay=0.01,
)


import torch
from transformers import Trainer

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss


trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics_for_regression,
)


if train:
    ## TRAINING
    t = time()

    trainer.train()
    trainer.save_model(trained_model_name)
    tokenizer.save_pretrained(trained_model_name)

    print('Time to train model: {} mins'.format(round((time() - t) / 60, 2)))

else:
    ## EVALUATION
    t = time()

    trainer.eval_dataset=ds["test"]
    metrics = trainer.evaluate()
    print(metrics)

    print('Time to eval model: {} mins'.format(round((time() - t) / 60, 2)))

# from local folder
# model = AutoModelForSequenceClassification.from_pretrained("./saved_model/")

# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# sequence = "mf"
# tokens = tokenizer.tokenize(sequence)

# print(tokens)

# ids = tokenizer.convert_tokens_to_ids(tokens)

# print(ids)

# import torch
# print(torch.cuda.is_available())
# torch.zeros(1).cuda()
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# print(device)



# import torch

# print(torch.cuda.is_available())

# print(torch.cuda.device_count())

# print(torch.cuda.current_device())

# print(torch.cuda.device(0))

# print(torch.cuda.get_device_name(0))

# Before running model_covid_twitter_bert_training.py
# True
# 1
# 0
# <torch.cuda.device object at 0x7f3f7bfb5de0>
# NVIDIA GeForce RTX 2070 SUPER

Didn't find file ./model_covid_twitter_bert_v2_trainer.model/added_tokens.json. We won't load it.
loading file ./model_covid_twitter_bert_v2_trainer.model/vocab.txt
loading file ./model_covid_twitter_bert_v2_trainer.model/tokenizer.json
loading file None
loading file ./model_covid_twitter_bert_v2_trainer.model/special_tokens_map.json
loading file ./model_covid_twitter_bert_v2_trainer.model/tokenizer_config.json
loading configuration file ./model_covid_twitter_bert_v2_trainer.model/config.json
Model config BertConfig {
  "_name_or_path": "./model_covid_twitter_bert_v2_trainer.model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "m

X.shape: (90, 2)
X.head():    created_at                                   tokenized_tweets
0  2021-01-01  {'input_ids': [101, 1049, 2546, 12677, 2979, 8...
1  2021-01-02  {'input_ids': [101, 5554, 2535, 2132, 2522, 17...
2  2021-01-03  {'input_ids': [101, 2293, 13055, 10859, 14045,...
3  2021-01-04  {'input_ids': [101, 2522, 17258, 2681, 2902, 5...
4  2021-01-05  {'input_ids': [101, 2522, 17258, 8778, 5658, 2...
train_df
train_df.shape: (62, 5)
    created_at                                   tokenized_tweets        date  \
0   2021-01-01  {'input_ids': [101, 1049, 2546, 12677, 2979, 8...  2021-01-01   
1   2021-01-02  {'input_ids': [101, 5554, 2535, 2132, 2522, 17...  2021-01-02   
2   2021-01-03  {'input_ids': [101, 2293, 13055, 10859, 14045,...  2021-01-03   
3   2021-01-04  {'input_ids': [101, 2522, 17258, 2681, 2902, 5...  2021-01-04   
4   2021-01-05  {'input_ids': [101, 2522, 17258, 8778, 5658, 2...  2021-01-05   
..         ...                                                ..

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

examples['tokenized_tweets']
{'input_ids': [101, 3909, 2710, 3073, 6947, 9398, 8382, 3231, 2765, 2604, 3462, 4553, 22950, 8398, 5658, 4276, 22088, 11368, 2098, 2343, 16985, 24014, 3672, 15121, 14132, 3478, 6960, 9424, 27860, 4297, 17572, 4928, 15011, 21122, 11533, 2522, 17258, 3433, 2111, 14753, 11451, 2522, 17258, 14855, 2497, 5396, 2318, 5926, 2304, 2111, 3404, 17404, 3369, 2057, 3726, 25216, 7272, 2317, 2304, 3424, 3567, 2595, 4895, 3995, 23062, 3085, 4357, 12484, 12849, 13344, 2050, 9413, 6371, 2304, 11376, 3189, 2522, 17258, 15050, 6545, 16464, 4945, 3460, 2703, 2175, 2361, 2375, 8571, 20221, 4909, 21887, 23350, 17404, 3019, 15403, 6526, 2147, 3946, 3044, 22950, 3514, 3006, 2709, 3671, 2522, 17258, 19428, 29464, 2050, 5157, 2180, 2102, 8980, 6090, 3207, 7712, 2095, 3930, 4030, 2522, 17258, 17404, 5219, 9432, 2233, 7610, 3693, 7484, 2592, 2389, 5219, 9298, 2080, 16184, 2319, 21414, 4654, 2102, 3693, 7396, 7411, 2522, 17258, 5604, 2194, 11891, 2571, 3231, 2354, 2338, 8814, 4819, 218

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 14
  Batch size = 2


ds
{'train': Dataset({
    features: ['created_at', 'tokenized_tweets', 'date', 'total_cases', 'g_values', '__index_level_0__'],
    num_rows: 62
}), 'validation': Dataset({
    features: ['created_at', 'tokenized_tweets', 'date', 'total_cases', 'g_values', '__index_level_0__'],
    num_rows: 14
}), 'test': Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 14
})}


{'eval_loss': 4.777312755584717, 'eval_mse': 4.777313232421875, 'eval_mae': 2.1771533489227295, 'eval_r2': -127.02297586879067, 'eval_accuracy': 0.0, 'eval_runtime': 1.6718, 'eval_samples_per_second': 8.374, 'eval_steps_per_second': 4.187}
Time to eval model: 0.03 mins
